# Imports

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

# Functions

In [ ]:
def print_stats(df, col):
    data = df.iloc[:, col]
    max_val = data.max()
    min_val = data.min()
    std_val = data.std()

    name = df.columns[col]
    ul = len(name) * '-'
    print(f"{name}")
    print(ul)
    print(f"max: {max_val}")
    print(f"min: {min_val}")
    print(f"std: {std_val}\n")
#-------------------------------------------------------------------------------

def hist_flow(
    ts_flow: pd.DataFrame,
    cols: list,
    title,
    names = ['Venturi', 'Nivus'],
    ):
    fig = make_subplots(rows=1, cols=len(cols), shared_yaxes=True)

    for i, col in enumerate(cols):
        fig.add_histogram(x=ts_flow.iloc[:,col], row=1, col=i+1, name=names[i])

    fig.update_layout(title_text=title)
    
    return fig

# Parameters

In [ ]:
path_data = Path(r"Q:\Projekte\8000-\8500er\8532\8532.14 Richterswil Beratung "
    r"Aussenbauwerke\05 Berechnungen Grundlagen\Zulauf Nivus-Venturi"
    r"\Daten_Nivus-Venturi.xlsx")
save_dir = Path(r"Q:\Projekte\8000-\8500er\8532\8532.14 Richterswil Beratung "
    r"Aussenbauwerke\05 Berechnungen Grundlagen\Zulauf Nivus-Venturi")

# Read Data

In [ ]:
data = pd.read_excel(path_data)
data.set_index(data.columns[0], inplace=True)

In [ ]:
# mask negative values
mask = data < 0
data[mask] = float('nan')

# mask unplausible values
mask = data > 150
data[mask] = float('nan')

# Plots

## Comparison Nivus / Venturi

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,0], name='Venturi'))
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,1], name='Nivus'))

fig.update_xaxes(title='Datum')
fig.update_yaxes(title='Durchfluss [l/s]')

fig.update_layout(
    title='Zulauf Berg',
    font=dict(size=25)
)

In [ ]:
save_path = save_dir / 'Zulauf-Berg.html'
fig.write_html(save_path)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,2], name='Venturi'))
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,3], name='Nivus'))

fig.update_xaxes(title='Datum')
fig.update_yaxes(title='Durchfluss [l/s]')

fig.update_layout(
    title='Zulauf See',
    font=dict(size=25)
)

In [ ]:
save_path = save_dir / 'Zulauf-See.html'
fig.write_html(save_path)

## Comparison Berg / See

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
    subplot_titles=['Zulauf Venturi', 'Zulauf Nivus'])

fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,0], name='Venturi Berg'), 
    row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,2], name='Venturi See'),
    row=1, col=1)

fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,1], name='Nivus Berg'),
    row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,3], name='Nivus See'),
    row=2, col=1)

fig.update_xaxes(title='Datum', row=2, col=1)
fig.update_yaxes(title='Durchfluss [l/s]')

fig.update_layout(
    font=dict(size=25)
)

fig.update_annotations(
    font_size=28
)

In [ ]:
save_path = save_dir / 'Zulauf_Venturi-Nivus.html'
fig.write_html(save_path)

## Histrograms

In [ ]:
# Berg
fig = hist_flow(data, cols=[0, 1], title='Histogramm Berg')
fig.update_layout(
    font=dict(size=25)
)

In [ ]:
save_path = save_dir / 'Histogramm-Berg.html'
fig.write_html(save_path)

In [ ]:
# See
fig = hist_flow(data, cols=[2, 3], title='Histogramm See')
fig.update_layout(
    font=dict(size=25)
)

In [ ]:
save_path = save_dir / 'Histogramm-See.html'
fig.write_html(save_path)

# Stats

In [ ]:
for i in range(len(data.columns)):
    print_stats(data, i)